# Цены на продукты питания

In [150]:
import numpy as np
import pandas as pd
import scipy
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import plotly
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import docx
import datetime as dt
import pickle
import random
import math
import time
import os
import re
import colorsys
import arrow
import locale
import requests
import collections
import numbers
import decimal

from warnings import simplefilter

from scipy import stats
from scipy.interpolate import make_interp_spline, BSpline
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from matplotlib.lines import Line2D
import matplotlib.dates as mdates

In [151]:
path_current = os.getcwd()
path_main = os.path.sep.join(path_current.split(os.path.sep)[:-2])

In [152]:
path_current

'/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus-dash/notebooks/data-prepare'

In [153]:
path_main

'/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus-dash'

### To main directory

In [154]:
# change directory to main path
os.chdir(path_main)

### Load libraries, functions, palette, theme

In [155]:
# load libraries and palette
%run __theme.ipynb

In [156]:
# load functions
# from __functions import *
%run __functions.ipynb

In [157]:
# start stopwatch
start = stopwatch_start()

### Variables

In [158]:
path_img = 'docs/img/'

In [159]:
path_files = path_main + '/files'

In [160]:
path_excel_csv = path_main + '/excel-csv'

### Load Files

In [161]:
economics_data = loadit('economics_data', path=path_files, create_empty_dict=True)

In [162]:
income = economics_data['households_income'].copy()

### Back to notebook directory

In [163]:
# back to current path
os.chdir(path_current)

# Домохозяйства

In [164]:
income_plot_data = income[income['Тип'] == 'Всего'].T[1:].iloc[:, -2:].copy()

In [165]:
income_plot_data = income_plot_data.astype(float)

In [166]:
income_plot_data['delta'] = \
    income_plot_data.iloc[:, 0] - income_plot_data.iloc[:, 1]

In [167]:
income_plot_data['delta_town'] = \
    (income_plot_data['Домашние хозяйства, проживающиев городской местности']
     .pct_change()
     .mul(100)
     .round(2))

income_plot_data['delta_country'] = \
    (income_plot_data['Домашние хозяйства, проживающиев сельской местности']
     .pct_change()
     .mul(100)
     .round(2))

In [168]:
income_plot_data['delta_pct'] = \
    income_plot_data['delta_town'] - income_plot_data['delta_country']

In [169]:
income_plot_data['country/town'] = \
    (income_plot_data['Домашние хозяйства, проживающиев сельской местности']
     / income_plot_data['Домашние хозяйства, проживающиев городской местности'])

In [170]:
income_plot_data['country/town'] = income_plot_data['country/town'].round(2)

In [171]:
income_plot_data.index = income_plot_data.index.astype(int)

In [172]:
income_plot_data

,"Домашние хозяйства, проживающиев городской местности","Домашние хозяйства, проживающиев сельской местности",delta,delta_town,delta_country,delta_pct,country/town
2003,4009.0,2423.2,1585.8,NaN,NaN,NaN,0.60
2004,5016.0,2851.3,2164.7,25.12,17.67,7.45,0.57
2005,6529.5,3604.7,2924.8,30.17,26.42,3.75,0.55
2006,7984.9,4495.4,3489.5,22.29,24.71,-2.42,0.56
2007,10354.6,5871.1,4483.5,29.68,30.60,-0.92,0.57
2008,13465.9,7786.5,5679.4,30.05,32.62,-2.57,0.58
2009,13869.4,8416.9,5452.5,3.00,8.10,-5.10,0.61
2010,16265.0,10128.7,6136.3,17.27,20.34,-3.07,0.62
2011,18291.1,11745.8,6545.3,12.46,15.97,-3.51,0.64
2012,20405.0,13320.3,7084.7,11.56,13.40,-1.84,0.65


### Save Dataset

In [173]:
economics_data['households_income_difference'] = income_plot_data

In [174]:
saveit(
    file=economics_data,
    name='economics_data',
    path=path_files
)

File 'economics_data.pkl' saved


### Execution time

In [175]:
stopwatch_stop(start)

Execution time: 0:00:00
